# Elabastecedor

## Prepare

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

from datetime import date
from pathlib import Path

In [2]:
# Get datasets

path = Path("data")
entries = [path / entry for entry in os.listdir(path) if entry.endswith(".csv")]
entries.sort()
list(entries)

[PosixPath('data/20231231_products.csv'),
 PosixPath('data/20240104_products.csv'),
 PosixPath('data/20240107_products.csv'),
 PosixPath('data/20240111_products.csv'),
 PosixPath('data/20240114_products.csv'),
 PosixPath('data/20240117_products.csv'),
 PosixPath('data/20240121_products.csv'),
 PosixPath('data/20240124_products.csv'),
 PosixPath('data/20240128_products.csv'),
 PosixPath('data/20240131_products.csv'),
 PosixPath('data/20240204_products.csv'),
 PosixPath('data/20240207_products.csv'),
 PosixPath('data/20240211_products.csv'),
 PosixPath('data/20240214_products.csv'),
 PosixPath('data/20240218_products.csv'),
 PosixPath('data/20240221_products.csv'),
 PosixPath('data/20240225_products.csv'),
 PosixPath('data/20240228_products.csv'),
 PosixPath('data/20240303_products.csv'),
 PosixPath('data/20240306_products.csv'),
 PosixPath('data/20240310_products.csv'),
 PosixPath('data/20240313_products.csv'),
 PosixPath('data/20240317_products.csv'),
 PosixPath('data/20240320_products

In [3]:
# Load datasets

# Get date from filename
def get_date(filename):
    year = filename[0:4]
    month = filename[4:6]
    day = filename[6:8]
    return f"{year}-{month}-{day}"

# Append datasets in a list
dfs = []
for file in entries:
    date = get_date(file.name)
    df = pd.read_csv(file, index_col="id")
    df.drop_duplicates(inplace=True)
    df.rename(columns={"precio":date}, inplace=True)
    dfs.append(df)

len(dfs)

59

In [4]:
# Merge datasets

df = pd.concat(dfs)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154855 entries, 44 to 61
Data columns (total 63 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   codigo      154855 non-null  int64  
 1   marca       154855 non-null  object 
 2   nombre      154855 non-null  object 
 3   2023-12-31  75 non-null      float64
 4   2024-01-04  73 non-null      float64
 5   2024-01-07  84 non-null      float64
 6   2024-01-11  3124 non-null    float64
 7   categoria   154623 non-null  object 
 8   2024-01-14  3103 non-null    float64
 9   2024-01-17  3033 non-null    float64
 10  2024-01-21  3067 non-null    float64
 11  2024-01-24  2978 non-null    float64
 12  2024-01-28  3034 non-null    float64
 13  2024-01-31  2946 non-null    float64
 14  2024-02-04  3082 non-null    float64
 15  2024-02-07  3062 non-null    float64
 16  2024-02-11  3102 non-null    float64
 17  2024-02-14  2988 non-null    float64
 18  2024-02-18  3080 non-null    float64
 19  2024-02-21

## Analyze

### Categories

In [5]:
# Get categories

categories = df[["codigo", "categoria"]].dropna().drop_duplicates(["codigo"]).set_index("codigo")
categories

,categoria
codigo,
114746,ACEITES
112222,ACEITES
103100,ACEITES
112537,ACEITES
114573,ACEITES
...,...
124286,PASTAS SECAS
82000,HAMBURGUESAS Y MEDALLONES
150697,GASEOSAS


### Products

In [6]:
# Get products

products = df[["codigo", "nombre", "marca"]].dropna().drop_duplicates(["codigo"]).set_index("codigo")
products["categoria"] = categories
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000 entries, 59 to 150647
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nombre     4000 non-null   object
 1   marca      4000 non-null   object
 2   categoria  3994 non-null   object
dtypes: object(3)
memory usage: 125.0+ KB


In [7]:
# Show products with missing category
products[products.categoria.isna()]

,nombre,marca,categoria
codigo,,,
27,BOLA DE LOMO XKG,A BALANZA,NaN
3034,MELON ROCIO XUNIDAD,A BALANZA,NaN
67000,CAJA NAVIDEÃA PREMIUM AZUL,BON MASE,NaN
3026,MANDARINA MURCOTXKG,A BALANZA,NaN
123525,TIERNITOS CARNE X1.5KG,INVENTARIABLES,NaN
123523,TIERNITOS CACHORROS C/L/C X1.5KG,EL ABASTECEDOR,NaN


In [8]:
# Drop products with missing category

products.dropna(inplace=True)
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3994 entries, 59 to 150647
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nombre     3994 non-null   object
 1   marca      3994 non-null   object
 2   categoria  3994 non-null   object
dtypes: object(3)
memory usage: 124.8+ KB


### Prices

In [9]:
# Get prices
prices = (df
           .groupby("codigo")
           .sum()
           .replace(0, np.NaN)
           .drop(columns=products.columns))
prices

,2023-12-31,2024-01-04,2024-01-07,2024-01-11,2024-01-14,2024-01-17,2024-01-21,2024-01-24,2024-01-28,2024-01-31,...,2024-06-19,2024-06-23,2024-06-26,2024-06-30,2024-07-03,2024-07-07,2024-07-10,2024-07-14,2024-07-17,2024-07-21
codigo,,,,,,,,,,,,,,,,,,,,,
2,5150.0,4099.0,4099.0,3699.0,NaN,3699.0,3699.0,3699.0,4849.0,5449.0,...,6199.0,6199.0,6199.0,6199.0,NaN,6199.0,6199.0,6199.0,6199.0,5899.0
3,5600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5850.0,4799.0,4799.0,4399.0,4399.0,4399.0,4399.0,4399.0,4999.0,4999.0,...,5999.0,5999.0,5999.0,6899.0,6899.0,6899.0,6899.0,6899.0,6899.0,5999.0
7,7300.0,5899.0,5399.0,5399.0,5399.0,5399.0,5399.0,5399.0,6799.0,7299.0,...,8399.0,8399.0,8399.0,7999.0,7999.0,8399.0,8399.0,NaN,7999.0,7999.0
11,9800.0,7999.0,7999.0,7999.0,7999.0,7999.0,7999.0,NaN,7999.0,7999.0,...,7299.0,7299.0,7299.0,9449.0,9449.0,9449.0,9449.0,NaN,9449.0,9449.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4999.0
150649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549.0,NaN,1549.0
150650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1005.0,1040.0,1040.0


In [10]:
# Show prices' statistics per date
prices.describe()

,2023-12-31,2024-01-04,2024-01-07,2024-01-11,2024-01-14,2024-01-17,2024-01-21,2024-01-24,2024-01-28,2024-01-31,...,2024-06-19,2024-06-23,2024-06-26,2024-06-30,2024-07-03,2024-07-07,2024-07-10,2024-07-14,2024-07-17,2024-07-21
count,75.000000,73.000000,84.000000,3124.000000,3103.000000,3033.000000,3067.000000,2978.000000,3034.000000,2946.000000,...,2486.000000,2503.000000,2453.000000,2503.000000,2418.000000,2435.000000,2354.000000,2391.000000,2333.000000,2374.000000
mean,3050.386667,2872.684932,2476.964286,2496.435915,2530.728073,2621.589149,2618.719677,2688.513227,2632.066882,2671.545815,...,3589.733166,3605.211850,3612.376351,3603.830823,3639.468875,3624.222066,3669.463267,3640.394178,3687.599511,3660.047224
std,2810.353228,2641.349344,2445.581634,3496.646721,3448.710021,3734.677710,3655.598256,3700.222567,3641.347483,3677.161676,...,6030.810552,5970.957706,6027.584702,6025.722272,6114.885173,5848.591237,5937.416234,5931.036778,5976.974425,5900.297844
min,90.000000,90.000000,120.000000,59.990000,59.990000,59.990000,110.000000,110.000000,110.000000,120.000000,...,180.000000,149.000000,149.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,149.000000
25%,749.000000,799.000000,736.500000,987.250000,999.000000,1030.000000,1027.500000,1079.000000,1045.000000,1069.000000,...,1300.000000,1312.000000,1310.000000,1329.000000,1325.000000,1325.000000,1325.000000,1324.500000,1340.000000,1349.000000
50%,1600.000000,1719.000000,1394.500000,1536.000000,1599.000000,1650.000000,1690.000000,1725.500000,1710.000000,1749.000000,...,2102.500000,2149.000000,2155.000000,2155.000000,2160.000000,2155.000000,2162.500000,2174.000000,2192.000000,2160.000000
75%,5175.000000,4799.000000,4141.500000,2700.000000,2822.000000,2930.000000,2889.000000,2990.000000,2839.000000,2915.000000,...,3732.500000,3771.000000,3745.000000,3730.000000,3718.750000,3700.000000,3799.000000,3737.500000,3819.000000,3821.250000
max,9999.000000,9999.000000,12000.000000,71415.000000,71415.000000,90483.000000,79839.000000,79839.000000,79839.000000,79839.000000,...,128237.000000,119999.000000,119999.000000,128237.000000,128236.000000,128236.000000,128236.000000,128236.000000,128236.000000,128236.000000


In [11]:
# Show price's statistics per product
prices_per_product = prices.T
prices_per_product.describe()

codigo,2,3,4,7,11,12,13,15,16,17,...,150636,150637,150639,150642,150646,150647,150649,150650,150664,150697
count,57.000000,7.000000,58.000000,58.000000,56.000000,52.000000,7.0,55.000000,53.000000,58.000000,...,5.0,3.0,10.0,1.0,5.000000,1.0,2.0,3.000000,1.0,1.0
mean,5584.982456,6199.142857,6006.775862,7511.086207,8632.946429,6135.538462,11349.0,2597.181818,2012.207547,6423.172414,...,1099.0,3145.0,2100.0,3349.0,1655.600000,4999.0,1549.0,1028.333333,2360.0,1999.0
std,683.516394,395.558857,836.856001,882.789921,735.933160,725.185263,0.0,696.682857,402.892301,875.624211,...,0.0,0.0,0.0,NaN,372.791899,NaN,0.0,20.207259,NaN,NaN
min,3699.000000,5600.000000,4399.000000,5399.000000,6999.000000,4999.000000,11349.0,1599.000000,1399.000000,4299.000000,...,1099.0,3145.0,2100.0,3349.0,1299.000000,4999.0,1549.0,1005.000000,2360.0,1999.0
25%,5449.000000,5999.000000,5399.000000,7199.000000,7999.000000,5399.000000,11349.0,1999.000000,1699.000000,5999.000000,...,1099.0,3145.0,2100.0,3349.0,1299.000000,4999.0,1549.0,1022.500000,2360.0,1999.0
50%,5699.000000,6199.000000,5999.000000,7999.000000,8499.000000,6499.000000,11349.0,2999.000000,2199.000000,6699.000000,...,1099.0,3145.0,2100.0,3349.0,1745.000000,4999.0,1549.0,1040.000000,2360.0,1999.0
75%,5899.000000,6449.000000,6899.000000,7999.000000,9449.000000,6899.000000,11349.0,2999.000000,2199.000000,7199.000000,...,1099.0,3145.0,2100.0,3349.0,1745.000000,4999.0,1549.0,1040.000000,2360.0,1999.0
max,6199.000000,6699.000000,6899.000000,8399.000000,9800.000000,6899.000000,11349.0,4199.000000,2999.000000,7199.000000,...,1099.0,3145.0,2100.0,3349.0,2190.000000,4999.0,1549.0,1040.000000,2360.0,1999.0


In [12]:
# price change per date
change_per_date = prices_per_product.pct_change(fill_method=None) * 100
change_per_date

codigo,2,3,4,7,11,12,13,15,16,17,...,150636,150637,150639,150642,150646,150647,150649,150650,150664,150697
2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,-20.407767,NaN,-17.965812,-19.191781,-18.377551,NaN,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-07,0.000000,NaN,0.000000,-8.476013,0.000000,NaN,NaN,NaN,NaN,-14.303571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-11,-9.758478,NaN,-8.335070,0.000000,0.000000,NaN,NaN,NaN,NaN,-10.418837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-14,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-17,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-21,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-24,0.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,9.380863,7.147963,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-28,31.089484,NaN,13.639464,25.930728,NaN,0.000000,NaN,0.000000,0.000000,27.913468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-31,12.373685,NaN,0.000000,7.354023,0.000000,0.000000,NaN,5.717553,6.671114,9.092562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Show prices' statistics per date
change_per_date.T.describe()

,2023-12-31,2024-01-04,2024-01-07,2024-01-11,2024-01-14,2024-01-17,2024-01-21,2024-01-24,2024-01-28,2024-01-31,...,2024-06-19,2024-06-23,2024-06-26,2024-06-30,2024-07-03,2024-07-07,2024-07-10,2024-07-14,2024-07-17,2024-07-21
count,0.0,67.000000,53.000000,78.000000,3049.000000,3002.000000,2948.000000,2969.000000,2901.000000,2922.000000,...,2466.000000,2431.000000,2425.000000,2401.000000,2394.000000,2348.000000,2325.000000,2295.000000,2296.000000,2258.000000
mean,NaN,3.378456,-0.785705,8.438657,3.891930,2.225437,2.088244,2.612687,-0.185708,0.781613,...,-0.070846,1.427801,0.050200,1.005317,0.776035,0.667216,0.220152,0.315560,0.335020,0.556514
std,NaN,16.987697,17.008046,22.316726,16.749631,10.859529,14.541099,10.953936,10.865165,8.816803,...,4.752733,21.047452,10.644397,8.781132,6.797026,8.674701,2.508991,8.559935,4.482375,8.479924
min,NaN,-20.407767,-42.424242,-42.892857,-42.074776,-40.016006,-50.050000,-35.714286,-64.428571,-63.660899,...,-92.113454,-76.692231,-60.891304,-82.775864,-75.041667,-54.094109,-22.666667,-43.362241,-35.001001,-50.166667
25%,NaN,0.000000,-7.846154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,0.000000,0.000000,17.809986,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,NaN,55.833333,61.616162,105.444444,174.413457,124.068966,353.805073,177.570093,168.168168,204.554697,...,38.196629,900.500278,480.580645,76.529477,83.402001,115.012510,60.080040,116.971369,69.154229,71.673820


In [14]:
def pct_change(key, values, df):
    "Group by dataset, get pct_change from mean"
    return (df
     .assign(**{key: values})
     .groupby(key)
     .mean()
     .pct_change(fill_method=None) * 100)

In [15]:
# change per month

key = "Mes"
values = [column[0:7] for column in prices.columns]
change_per_month = pct_change(key, values, prices_per_product)
change_per_month

codigo,2,3,4,7,11,12,13,15,16,17,...,150636,150637,150639,150642,150646,150647,150649,150650,150664,150697
Mes,,,,,,,,,,,,,,,,,,,,,
2023-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01,-19.194175,NaN,-21.004748,-20.257230,-18.377551,NaN,NaN,NaN,NaN,-13.904762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02,33.191157,NaN,19.535476,24.527113,3.125391,10.502100,NaN,13.571915,12.385951,23.907633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03,2.217612,NaN,9.403411,3.602029,4.512331,10.610266,NaN,13.668372,14.003463,7.672135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04,4.339001,NaN,7.744847,5.178204,5.541880,6.569257,NaN,30.398646,15.186707,6.866352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05,-1.903070,NaN,-1.535744,2.391301,-6.105677,1.855691,0.0,6.196884,2.326664,3.636893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06,4.215286,NaN,1.364735,1.099037,1.162360,-2.010353,0.0,0.000000,0.000000,1.052779,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07,1.581144,NaN,3.846746,-0.217418,9.328628,6.154793,NaN,32.010670,29.104138,-4.861786,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Show prices' statistics per month
change_per_month.T.describe()

Mes,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07
count,0.0,72.000000,3116.000000,3165.000000,3054.000000,2845.000000,2652.000000,2527.000000
mean,NaN,9.666119,8.348681,6.641454,5.329718,2.399522,2.514801,2.368459
std,NaN,27.031032,21.276038,13.862161,12.220355,11.753905,10.050030,9.080667
min,NaN,-42.546784,-60.025447,-47.826087,-37.061256,-60.534195,-41.818182,-73.179104
25%,NaN,-9.326777,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,0.490870,3.544087,3.592085,2.009306,0.000000,0.682136,0.896861
75%,NaN,29.171531,12.872996,10.809463,9.239517,4.575163,5.155639,5.340285
max,NaN,87.945434,275.000000,168.000000,150.000000,258.333333,190.019002,123.027068


In [17]:
# change per quarter

def calc_quarter(date):
    y = date[0:4]
    m = date[5:7]
    q = ((int(m) - 1) // 3) + 1
    return f"{y}-{q}"

key = "Trimestre"
values = [calc_quarter(date) for date in prices.columns]
change_per_quarter = pct_change(key, values, prices_per_product)
change_per_quarter

codigo,2,3,4,7,11,12,13,15,16,17,...,150636,150637,150639,150642,150646,150647,150649,150650,150664,150697
Trimestre,,,,,,,,,,,,,,,,,,,,,
2023-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-1,-0.097087,12.482143,-7.840894,-6.230769,-15.275510,NaN,NaN,NaN,NaN,2.386676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-2,15.028781,NaN,20.100732,17.755402,5.419728,16.947241,NaN,52.086437,30.691686,23.324501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-3,3.730325,NaN,4.231660,1.221526,7.951559,5.519461,NaN,34.424217,30.090118,-3.139584,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Show prices' statistics per quarter
change_per_quarter.T.describe()

Trimestre,2023-4,2024-1,2024-2,2024-3
count,0.0,73.000000,3200.000000,2578.000000
mean,NaN,38.316990,14.339887,4.316489
std,NaN,56.364365,24.413136,10.948162
min,NaN,-39.742105,-62.812400,-73.106545
25%,NaN,-0.647929,1.718361,0.000000
50%,NaN,18.049410,9.595403,3.832792
75%,NaN,61.807808,22.579153,8.814091
max,NaN,223.645546,389.141068,107.910582


In [19]:
# change per year

key = "Año"
values = [date[0:4] for date in prices.columns]
change_per_year = pct_change(key, values, prices_per_product)
change_per_year

codigo,2,3,4,7,11,12,13,15,16,17,...,150636,150637,150639,150642,150646,150647,150649,150650,150664,150697
Año,,,,,,,,,,,,,,,,,,,,,
2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,8.597087,12.482143,2.726946,2.942322,-12.125232,NaN,NaN,NaN,NaN,14.957393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Show prices' statistics per year
change_per_year.T.describe()

Año,2023,2024
count,0.0,74.000000
mean,NaN,75.190788
std,NaN,108.467009
min,NaN,-18.357670
25%,NaN,6.894821
50%,NaN,30.870420
75%,NaN,94.434993
max,NaN,530.678571
